In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

In [2]:
BATCH_SIZE = 256
train_rate = 0.1

In [3]:
# load data without header
data = pd.read_csv('./ISWC21_data_plus_raw/rwhar_3sbjs_data.csv', header=None)
# add header
data.columns = ['subject', 'x', 'y', 'z', 'activity']
data.head()

,subject,x,y,z,activity
0,0,0.378284,10.168175,0.847547,climbing_up
1,0,0.383671,10.172364,0.849942,climbing_up
2,0,0.372298,10.181941,0.859518,climbing_up
3,0,0.342969,10.170568,0.834379,climbing_up
4,0,0.319626,10.159795,0.818817,climbing_up


In [4]:
data.shape

(659260, 5)

In [5]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['activity'])
data['encoded_activity'] = encoded_labels

In [6]:
data.head()

,subject,x,y,z,activity,encoded_activity
0,0,0.378284,10.168175,0.847547,climbing_up,1
1,0,0.383671,10.172364,0.849942,climbing_up,1
2,0,0.372298,10.181941,0.859518,climbing_up,1
3,0,0.342969,10.170568,0.834379,climbing_up,1
4,0,0.319626,10.159795,0.818817,climbing_up,1


In [7]:
def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array along with the mode label for each window.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows, indices, and labels
    """
    windows = []
    indices = []
    labels = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * win_len)
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        window = data[curr:curr + win_len]
        windows.append(window.iloc[:, :-2])  # Exclude the last two columns (original and encoded labels)
        indices.append([curr, curr + win_len])
        
        # Extract and compute the mode of the encoded labels for the current window
        window_labels = window['encoded_activity']
        mode_result = mode(window_labels)
        window_label = mode_result[0] if mode_result[0].size > 0 else mode_result
        labels.append(window_label)

        curr += win_len - overlapping_elements

    result_windows = np.array(windows)
    result_indices = np.array(indices)
    result_labels = np.array(labels)
    return result_windows, result_indices, result_labels


In [8]:
sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

window_data, _, window_label = sliding_window_samples(data, window_size, overlap_ratio)
print(f"shape of window dataset (2 sec with 0% overlap): {window_data.shape}")
print(f"shape of window label (2 sec with 0% overlap): {window_label.shape}")

shape of window dataset (2 sec with 0% overlap): (6592, 100, 4)
shape of window label (2 sec with 0% overlap): (6592,)


In [9]:
#remove the subject column
window_data = window_data[:, :, 1:]

In [10]:
window_data[0].shape

(100, 3)

In [11]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
import numpy as np


window_data = window_data.astype(np.float32)

# Convert to PyTorch tensors
window_data_tensor = torch.from_numpy(window_data)
window_label_tensor = torch.from_numpy(window_label)
#convert labels to long
window_label_tensor = window_label_tensor.long()

# split data into train and test sets
train_size = int(train_rate * len(window_data_tensor))
test_size = len(window_data_tensor) - train_size

# Creating datasets
dataset = TensorDataset(window_data_tensor, window_label_tensor)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Function to extract tensors from Subset
def extract_subset_data(subset, dataset):
    return dataset.tensors[0][subset.indices], dataset.tensors[1][subset.indices]

# Extract data and labels from train and test sets
train_data, train_labels = extract_subset_data(train_dataset, dataset)
test_data, test_labels = extract_subset_data(test_dataset, dataset)

# create train and test TensorDataset
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

# create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
#  print the shape of train_loader and test_loader
print(f"shape of train_loader: {len(train_loader)}")
print(f"shape of test_loader: {len(test_loader)}")

shape of train_loader: 3
shape of test_loader: 24


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        
        res = x if self.downsample is None else self.downsample(x)

        # Adjusting the length of the residual to match the output
        if out.size(2) != res.size(2):
            desired_length = out.size(2)
            res = res[:, :, :desired_length]

        return self.relu(out + res)


class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout=0.2, num_classes=4):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size + (dilation_size - 1))]

        self.tcn = nn.Sequential(*layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        x = self.tcn(x)
        x = F.avg_pool1d(x, x.size(2)).squeeze(2)  # Global Average Pooling
        x = self.dropout(x)
        return self.fc(x)

In [15]:
# create training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.transpose(1, 2)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [16]:
# create testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    #calculate accuracy
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.transpose(1, 2)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            #calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return running_loss / len(test_loader), accuracy

In [17]:
# create function to train and test model
def train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    test_losses = []
    test_accuracies = []
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        print(f"Epoch: {epoch + 1}/{num_epochs}.. Train Loss: {train_loss:.3f}.. Test Loss: {test_loss:.3f}.. Test Accuracy: {test_accuracy:.3f}")
    return train_losses, test_losses, test_accuracies

In [18]:
# get number of classes
num_classes = len(np.unique(window_label))
print(f"number of classes: {num_classes}")

number of classes: 8


In [19]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/50.. Train Loss: 1.877.. Test Loss: 1.589.. Test Accuracy: 0.446
Epoch: 2/50.. Train Loss: 1.392.. Test Loss: 1.225.. Test Accuracy: 0.514
Epoch: 3/50.. Train Loss: 1.114.. Test Loss: 1.060.. Test Accuracy: 0.591
Epoch: 4/50.. Train Loss: 0.980.. Test Loss: 0.949.. Test Accuracy: 0.691
Epoch: 5/50.. Train Loss: 0.839.. Test Loss: 0.862.. Test Accuracy: 0.714
Epoch: 6/50.. Train Loss: 0.760.. Test Loss: 0.829.. Test Accuracy: 0.729
Epoch: 7/50.. Train Loss: 0.685.. Test Loss: 0.802.. Test Accuracy: 0.780
Epoch: 8/50.. Train Loss: 0.678.. Test Loss: 0.883.. Test Accuracy: 0.700
Epoch: 9/50.. Train Loss: 0.683.. Test Loss: 0.809.. Test Accuracy: 0.754
Epoch: 10/50.. Train Loss: 0.605.. Test Loss: 0.766.. Test Accuracy: 0.752
Epoch: 11/50.. Train Loss: 0.599.. Test Loss: 0.712.. Test Accuracy: 0.774
Epoch: 12/50.. Train Loss: 0.528.. Test Loss: 0.703.. Test Accuracy: 0.786
Epoch: 13/50.. Train Loss: 0.500.. Test Loss: 0.689.. Test Accuracy: 0.789
Epoch: 14/50.. Train Loss: 0.458..

In [20]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)


Epoch: 1/50.. Train Loss: 2.535.. Test Loss: 2.514.. Test Accuracy: 0.141
Epoch: 2/50.. Train Loss: 2.443.. Test Loss: 2.418.. Test Accuracy: 0.147
Epoch: 3/50.. Train Loss: 2.345.. Test Loss: 2.329.. Test Accuracy: 0.146
Epoch: 4/50.. Train Loss: 2.273.. Test Loss: 2.245.. Test Accuracy: 0.142
Epoch: 5/50.. Train Loss: 2.194.. Test Loss: 2.166.. Test Accuracy: 0.141
Epoch: 6/50.. Train Loss: 2.126.. Test Loss: 2.093.. Test Accuracy: 0.215
Epoch: 7/50.. Train Loss: 2.053.. Test Loss: 2.025.. Test Accuracy: 0.266
Epoch: 8/50.. Train Loss: 1.991.. Test Loss: 1.965.. Test Accuracy: 0.275
Epoch: 9/50.. Train Loss: 1.932.. Test Loss: 1.911.. Test Accuracy: 0.282
Epoch: 10/50.. Train Loss: 1.875.. Test Loss: 1.863.. Test Accuracy: 0.334
Epoch: 11/50.. Train Loss: 1.833.. Test Loss: 1.820.. Test Accuracy: 0.358
Epoch: 12/50.. Train Loss: 1.797.. Test Loss: 1.783.. Test Accuracy: 0.369
Epoch: 13/50.. Train Loss: 1.767.. Test Loss: 1.750.. Test Accuracy: 0.379
Epoch: 14/50.. Train Loss: 1.734..

In [21]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

# Update the final classification layer
model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)

# Move the model to the appropriate device
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer (only update the parameters that require gradients)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Train and test the model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)


Epoch: 1/50.. Train Loss: 2.265.. Test Loss: 2.110.. Test Accuracy: 0.091
Epoch: 2/50.. Train Loss: 2.013.. Test Loss: 1.927.. Test Accuracy: 0.159
Epoch: 3/50.. Train Loss: 1.835.. Test Loss: 1.794.. Test Accuracy: 0.323
Epoch: 4/50.. Train Loss: 1.714.. Test Loss: 1.679.. Test Accuracy: 0.434
Epoch: 5/50.. Train Loss: 1.578.. Test Loss: 1.568.. Test Accuracy: 0.464
Epoch: 6/50.. Train Loss: 1.447.. Test Loss: 1.452.. Test Accuracy: 0.484
Epoch: 7/50.. Train Loss: 1.332.. Test Loss: 1.339.. Test Accuracy: 0.515
Epoch: 8/50.. Train Loss: 1.192.. Test Loss: 1.232.. Test Accuracy: 0.603
Epoch: 9/50.. Train Loss: 1.075.. Test Loss: 1.129.. Test Accuracy: 0.639
Epoch: 10/50.. Train Loss: 0.958.. Test Loss: 1.038.. Test Accuracy: 0.663
Epoch: 11/50.. Train Loss: 0.878.. Test Loss: 0.963.. Test Accuracy: 0.674
Epoch: 12/50.. Train Loss: 0.790.. Test Loss: 0.897.. Test Accuracy: 0.686
Epoch: 13/50.. Train Loss: 0.716.. Test Loss: 0.840.. Test Accuracy: 0.739
Epoch: 14/50.. Train Loss: 0.651..

In [22]:
num_inputs = 3  # Assuming 3 input channels (x, y, z axes of the accelerometer)
num_channels = [64, 128, 256]  # Example channel sizes for each layer
kernel_size = 8  # Kernel size for temporal convolutions

model = TCN(num_inputs, num_channels, kernel_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/50.. Train Loss: 1.875.. Test Loss: 1.391.. Test Accuracy: 0.565
Epoch: 2/50.. Train Loss: 1.353.. Test Loss: 1.092.. Test Accuracy: 0.615
Epoch: 3/50.. Train Loss: 1.106.. Test Loss: 0.989.. Test Accuracy: 0.610
Epoch: 4/50.. Train Loss: 0.969.. Test Loss: 0.943.. Test Accuracy: 0.675
Epoch: 5/50.. Train Loss: 0.863.. Test Loss: 0.887.. Test Accuracy: 0.705
Epoch: 6/50.. Train Loss: 0.814.. Test Loss: 0.867.. Test Accuracy: 0.696
Epoch: 7/50.. Train Loss: 0.780.. Test Loss: 0.819.. Test Accuracy: 0.713
Epoch: 8/50.. Train Loss: 0.695.. Test Loss: 0.783.. Test Accuracy: 0.750
Epoch: 9/50.. Train Loss: 0.662.. Test Loss: 0.809.. Test Accuracy: 0.732
Epoch: 10/50.. Train Loss: 0.648.. Test Loss: 0.702.. Test Accuracy: 0.788
Epoch: 11/50.. Train Loss: 0.558.. Test Loss: 0.691.. Test Accuracy: 0.771
Epoch: 12/50.. Train Loss: 0.535.. Test Loss: 0.684.. Test Accuracy: 0.785
Epoch: 13/50.. Train Loss: 0.532.. Test Loss: 0.647.. Test Accuracy: 0.810
Epoch: 14/50.. Train Loss: 0.480..

In [23]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_20231216-0131.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)

Epoch: 1/50.. Train Loss: 2.145.. Test Loss: 2.068.. Test Accuracy: 0.146
Epoch: 2/50.. Train Loss: 2.068.. Test Loss: 2.013.. Test Accuracy: 0.169
Epoch: 3/50.. Train Loss: 2.004.. Test Loss: 1.965.. Test Accuracy: 0.210
Epoch: 4/50.. Train Loss: 1.949.. Test Loss: 1.923.. Test Accuracy: 0.266
Epoch: 5/50.. Train Loss: 1.917.. Test Loss: 1.886.. Test Accuracy: 0.327
Epoch: 6/50.. Train Loss: 1.873.. Test Loss: 1.852.. Test Accuracy: 0.331
Epoch: 7/50.. Train Loss: 1.841.. Test Loss: 1.821.. Test Accuracy: 0.340
Epoch: 8/50.. Train Loss: 1.809.. Test Loss: 1.791.. Test Accuracy: 0.343
Epoch: 9/50.. Train Loss: 1.784.. Test Loss: 1.763.. Test Accuracy: 0.341
Epoch: 10/50.. Train Loss: 1.770.. Test Loss: 1.737.. Test Accuracy: 0.390
Epoch: 11/50.. Train Loss: 1.708.. Test Loss: 1.713.. Test Accuracy: 0.391
Epoch: 12/50.. Train Loss: 1.702.. Test Loss: 1.689.. Test Accuracy: 0.392
Epoch: 13/50.. Train Loss: 1.679.. Test Loss: 1.667.. Test Accuracy: 0.399
Epoch: 14/50.. Train Loss: 1.644..